In [1]:
import numpy as np
import pandas as pd
import csv
import openpyxl
'''train = np.array([
    [0,0,1,2,3],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
]
)'''
'''
train = np.array([
    [0,1,1,1,3],
    [1,1,1,1,4],
    [2,1,1,1,5],
    [3,1,1,1,6]
]
)'''
train = pd.read_csv('train.csv').as_matrix()#read train.csv file
#print(type(train))
#xtrain=train[:,1:4]
#xtrain = xtrain/5
#true_label=train[:,4:5]
#print((xtrain),(true_label))


C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [2]:
def encode_into_prob(x,max_val):
    len=x.size
    y_list=[]
    for i in range(len):
        temp=[]
        for j in range(max_val):
            if(x[i]==j):
                temp.append(1)
            else:
                temp.append(0)
            #print(temp)
        y_list.append(temp)
    return y_list
        

In [3]:
xtrain=train[:,1:785]

xmean = np.mean(xtrain , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
xtrain = xtrain - means_expanded

xstd = np.std(xtrain , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
xtrain = xtrain*1.0/std_expanded 

true_label = train[:,785:]
y_true=np.array(encode_into_prob(true_label,10))
y_true=y_true.transpose()

#print(type(y_true))
#print(y_true.transpose())
print(xtrain)
print((y_true))

[[-0.55945675 -0.55945675  2.06254185 ...  2.0522595  -0.55945675
  -0.55945675]
 [-0.5380259  -0.5380259  -0.5380259  ... -0.5380259  -0.5380259
  -0.5380259 ]
 [-0.6486501  -0.6486501  -0.6486501  ... -0.6486501  -0.6486501
  -0.6486501 ]
 ...
 [-0.64973918 -0.64973918 -0.64973918 ... -0.64973918 -0.64973918
  -0.64973918]
 [-0.59745866 -0.59745866 -0.59745866 ... -0.59745866 -0.59745866
  -0.59745866]
 [-0.62629522 -0.62629522 -0.62629522 ... -0.62629522 -0.62629522
  -0.62629522]]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


In [4]:
def feedforward(batch_train,hidden_layers):
     
    global weight_matrix_list
    global bias_list
    global preactivation_list 
    preactivation_list=[]
    global activation_list
    activation_list=[]
    global y_hat
    y_hat=[]
    a_new=[]
    a_last_layer=[]
    nos_data = len(batch_train)
    #print(nos_data)
    h=batch_train.transpose()
    preactivation_list.append(h)
    activation_list.append(h)
    #print(h)
    
#     print("y_hat in feed\n",y_hat )
    
    for j in range(hidden_layers):
        
#         print(".............................")
#         print("activation for layer ", j)
#         print(activation_list[j] , activation_list[j].shape)
#         print("weight",j+1)
#         print(weight_matrix_list[j], weight_matrix_list[j].shape)
#         print("bias ",j+1)
#         print(bias_list[j],bias_list[j].shape)
        
        a_new = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),activation_list[j])
        preactivation_list.append(a_new)
        
#         print("preactivation for layer",j+1)
#         print(preactivation_list[j+1], preactivation_list[j+1].shape)
        
        h = 1.0/(1.0+np.exp(-1*preactivation_list[j+1]))
        activation_list.append(h)

    
    j=hidden_layers
#     print(".............................")
#     print("activation for layer ", j)
#     print(activation_list[j] , activation_list[j].shape)
#     print("weight",j+1)
#     print(weight_matrix_list[j], weight_matrix_list[j].shape)
#     print("bias ",j+1)
#     print(bias_list[j],bias_list[j].shape)
    
    a_last_layer = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),activation_list[j])
    preactivation_list.append(a_last_layer)
    
#     print("last activation")
#     print(a_last_layer,a_last_layer.shape)

    y_hat = np.array(output(a_last_layer,batch_train))
    y_hat = y_hat.transpose()
    
#     print("y_hat in feedforward")
#     print(y_hat, y_hat.shape)
    
    
def output(a_last_layer,batch_train):
    y_hat_initial=[]
    y_hat_local=[]
    nos_data=len(batch_train)
    for i in range(nos_data):
        y_hat_initial.append(softmax(a_last_layer[:,i:i+1].transpose()))

    length = len(y_hat_initial)
    for i in range(length):
        y_hat_local.append(y_hat_initial[i][0])
    
#     print("y_hat_local", y_hat_local)
#     print("y_hat_initial",y_hat_initial)
    return y_hat_local

def softmax(row_vector):
    sum_all=0
    row_vector = np.exp(row_vector)
    sum_all = np.sum(row_vector , axis=1)         #print("row_vector", row_vector)
    row_vector = (row_vector*1.0/sum_all)         #print("sum_all",sum_all)
    return row_vector

In [5]:
def backpropagation(y_true_batch):
    
#     print("weigt in backpropogation", weight_matrix_list)
#     print("...........y_true_batch..........\n",y_true_batch)
    
    global grad_a_k_list
    grad_a_k_list=[]
    
    global grad_w_k_list
    grad_w_k_list=[]
    
    global grad_h_k_list
    grad_h_k_list=[]
    
    global grad_b_k_list
    grad_b_k_list=[]
    
    global grad_b
    grad_b=[]
    
    global grad_g_dash_a_k_list
    grad_g_dash_a_k_list = []
    
    global grad_a_last
    grad_a_last=[]
    
    k = num_hidden+1
    
#     print("--------------------------in Backpropagation---------------------")
    
    grad_g_dash_a_k_list = g_dash(preactivation_list)
    
#     print("preactivation_list....................")
#     print("Nos. of matrices in preactivation_list is", len(preactivation_list))
#     print(preactivation_list)
#     print("g_dash..................")
#     print("Nos. of matrices in g_dash is ", len(grad_g_dash_a_k_list))
#     print(grad_g_dash_a_k_list)
    
    grad_a_last = -np.subtract(y_true_batch,y_hat)
    
#     print("y_true_batch \n", y_true_batch)
#     print("y_hat \n",y_hat)
    
    grad_a_k_list.insert(0,grad_a_last)
    
#     print("grad_a_last_layer", k)
#     print(grad_a_k_list[0],grad_a_k_list[0].shape)
        
    while(k>0):
        
#         print(".................grad_compute at layer",k)
        
        grad_w_k_list.insert(0,np.matmul(grad_a_k_list[0],preactivation_list[k-1].transpose()))
        
#         print("grad_a", k)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
#         print('preactivation  a_',k-1)
#         print(preactivation_list[k-1],preactivation_list[k-1].shape)
#         print("grad_w_",k)
#         print(grad_w_k_list[0],grad_w_k_list[0].shape)
        
        grad_b_k_list.insert(0,grad_a_k_list[0])
        
#         print("grad_b",k)
#         print(grad_b_k_list[0],grad_b_k_list[0].shape)
        
        grad_h_k_list.insert(0,np.matmul(grad_w_k_list[0].transpose(),grad_a_k_list[0]))
        
#         print("grad_w",k)
#         print(grad_w_k_list[0],grad_w_k_list[0].shape)
#         print("grad_a",k)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
#         print("grad_h",k-1)
#         print(grad_h_k_list[0],grad_h_k_list[0].shape)
        
        grad_a_k_list.insert(0,np.multiply(grad_h_k_list[0],grad_g_dash_a_k_list[k-1]))
        
#         print("grad_h_k_list[0]",k-1)
#         print(grad_h_k_list[0],grad_h_k_list[0].shape)
#         print("grad_g_dash_a_k_list[k-1]",k-1)
#         print(grad_g_dash_a_k_list[k-1],grad_g_dash_a_k_list[k-1].shape)
#         print("grad_a",k-1)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
        k=k-1
#         print("grad_a_ik_list",grad_a_k_list[0])
    
    sum_all_grad_b_of_data(grad_b_k_list)
    

def sum_all_grad_b_of_data(grad_b_k_list):
    length=len(grad_b_k_list)
    i=0
    while(i<length):
        row=[]
        row = np.sum(grad_b_k_list[i],1)
        j = len(row)
        #print(row.shape)
        row = row[:, np.newaxis]
        #print(row.shape , row)
        grad_b.append(row)
        i+=1
    #print("grad_b",grad_b)

def g_dash(preactivation_list):
    length=len(preactivation_list)
    a_modified = preactivation_list[0:length]
    g_dash_a_k_list=[]
    for i in range(length):
        a=(1.0/(1+np.exp(-a_modified[i])))*(1-(1.0/(1+np.exp(-a_modified[i]))))
        g_dash_a_k_list.append(a)
    return g_dash_a_k_list


In [6]:
def decode_yhat_to_classes(y_hat_local):
    col = len(y_hat_local[0])
    max_val=0
    for j in range(col):
        max_val = y_hat_local[0][j]
        index=0
        for i in range(9):
            if(y_hat_local[i+1][j] > max_val):
                y_hat_local[index][j]=0
                index = i+1
                max_val=y_hat[i+1][j]
            else :
                y_hat_local[i+1][j]=0
        y_hat_local[index][j]=1
    return y_hat_local

In [19]:
#ip_neurons = 3 #input layer
ip_neurons = 784
num_hidden = 0
encoding_bits=10
sizes = []
sizes.insert(0,ip_neurons)
sizes.append(encoding_bits)
print(sizes)
print(len(sizes))

weight_matrix_list = []

bias_list =[]
y_hat= []

for i in range(int(num_hidden)+1):
    weight_matrix_list.append(0.01*np.random.rand(sizes[i],sizes[i+1]))
    bias_list.append(0.01*np.random.rand(sizes[i+1],1))
print(weight_matrix_list)
#print(len(weight_matrix_list))

print(bias_list)
#print(hlist)



[784, 10]
2
[array([[0.00206275, 0.00359858, 0.00426711, ..., 0.00814957, 0.0043645 ,
        0.00522144],
       [0.00322303, 0.00379487, 0.00396095, ..., 0.00977087, 0.00795203,
        0.00294954],
       [0.0037125 , 0.00950683, 0.00829493, ..., 0.00747716, 0.00793004,
        0.00971369],
       ...,
       [0.00657197, 0.00621005, 0.00859556, ..., 0.00785171, 0.00750636,
        0.00413116],
       [0.00693964, 0.00417373, 0.00139809, ..., 0.00017605, 0.00818562,
        0.00525348],
       [0.0096623 , 0.00225102, 0.00329946, ..., 0.00168053, 0.00653391,
        0.00013677]])]
[array([[0.00538525],
       [0.0045405 ],
       [0.00454688],
       [0.00185866],
       [0.00171154],
       [0.00175973],
       [0.00323049],
       [0.00306376],
       [0.00686285],
       [0.00709521]])]


In [20]:
def nestrov_accelerated_gradient_descent(weight_matrix_list,bias_list,xtrain,num_hidden,sizes):
    
    prev_v_w = [] 
    prev_v_b = []
    for i in range(int(num_hidden)+1):
        prev_v_w.append(0*np.random.rand(sizes[i],sizes[i+1]))
        prev_v_b.append(0*np.random.rand(sizes[i+1],1))
    
    v_w = [] 
    v_b =[]
    for i in range(int(num_hidden)+1):
        v_w.append(0*np.random.rand(sizes[i],sizes[i+1]))
        v_b.append(0*np.random.rand(sizes[i+1],1))
        
    len_w = len(weight_matrix_list)  
    len_b = len(bias_list)
    
    eta = 0.001
    batch_size = 10
    steps = 0
    gamma = 0.9
    epochs = 10
    t=0
    data_length = len(xtrain)
    no_of_batch = int(data_length/batch_size)
    
    for t in range(epochs):
        for batch in range(no_of_batch):
            dw=[]
            db=[]
            for i in range(int(num_hidden)+1):
                dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
                db.append(0*np.random.rand(sizes[i+1],1))
                
            for j in range(len_w): 
                v_w[j] = gamma*prev_v_w[j]
                                    
            for j in range(len_b):
                v_b[j] = gamma*prev_v_b[j]
            
            for j in range(len_w):
                weight_matrix_list[j] -= v_w[j]
            
            for j in range(len_b):
                bias_list[j] -= v_b[j]
            
            for i in range(batch_size):
                l = batch_size*batch + i
                r = batch_size*batch + i + 1
                feedforward(xtrain[l:r,:],num_hidden)
                #print(l , r)
                backpropagation(y_true[:,l:r])

                for j in range(len_w): 
                    dw[j] += grad_w_k_list[j].transpose()
                                    
                for j in range(len_b):
                    db[j] += grad_b[j]
                                
            
            for j in range(len_w): 
                v_w[j] = gamma*prev_v_w[j] + eta*dw[j]
                                    
            for j in range(len_b):
                v_b[j] = gamma*prev_v_b[j] + eta*db[j]
                
            for j in range(len_w):
                weight_matrix_list[j] += -1*eta*dw[j]
            
            for j in range(len_b):
                bias_list[j] = -1*eta*db[j]
                
            for j in range(len_w): 
                prev_v_w[j] = v_w[j]
                                    
            for j in range(len_b):
                prev_v_b[j] = v_b[j]
                
            
        
        print("end of epoch" , t)

In [21]:
nestrov_accelerated_gradient_descent(weight_matrix_list,bias_list,xtrain,num_hidden,sizes)
#print(y_hat)0

end of epoch 0
end of epoch 1
end of epoch 2
end of epoch 3
end of epoch 4
end of epoch 5
end of epoch 6
end of epoch 7
end of epoch 8
end of epoch 9


In [22]:
print(y_hat)

[[4.40495942e-15]
 [2.21294782e-05]
 [1.86380810e-08]
 [1.66895552e-09]
 [2.02656640e-19]
 [1.18775787e-17]
 [9.99542288e-01]
 [2.43635880e-05]
 [1.29748374e-05]
 [3.98223983e-04]]


In [23]:
def decode_yhat_to_classes(y_hat):
    col = len(y_hat[0])
    max_val=0
    for j in range(col):
        max_val = y_hat[0][j]
        index=0
        for i in range(9):
            if(y_hat[i+1][j] > max_val):
                y_hat[index][j]=0
                index = i+1
                max_val=y_hat[i+1][j]
            else :
                y_hat[i+1][j]=0
        y_hat[index][j]=1
    return y_hat
xtest = pd.read_csv('test.csv').as_matrix()
test=xtest[:,1:]
feedforward(test,num_hidden)
#print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
#print(y_hat)

C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in true_divide


In [24]:
print(y_hat)

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [585]:
print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
print(y_hat)

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [603]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i+1)
                break
    return np.array(y_predict)

In [604]:
y_predict = classifcation(y_hat)
print((y_predict))
y_predict = y_predict[:, np.newaxis]
print((y_predict))

[4 4 4 ... 4 4 4]
[[4]
 [4]
 [4]
 ...
 [4]
 [4]
 [4]]


In [605]:
def add_id_to_y_predict(y_predict):
    ide=[]
    row=len(y_predict)
    for i in range(row):
        ide.append(i)
    ide=np.array(ide)
    ide=ide[:,np.newaxis]
    y= np.hstack((ide,y_predict))
    #print(y_predict, ide)
    return y           

In [606]:
print(y_predict)
#y_predict= np.random.randint(10, size=(10000, 1))
y_predict= add_id_to_y_predict(y_predict)
print(y_predict)
x,y = y_predict.shape
#print(x,y)

[[4]
 [4]
 [4]
 ...
 [4]
 [4]
 [4]]
[[   0    4]
 [   1    4]
 [   2    4]
 ...
 [9997    4]
 [9998    4]
 [9999    4]]


In [607]:
with open('predict.csv', 'w', newline='') as f:
    thewriter = csv.writer(f)
    row,col=y_predict.shape 
    thewriter.writerow(['id','label'])
    for i in range(row):
        thewriter.writerow(y_predict[i])
#out.close()
#np.savetxt("foo.csv", y_predict, delimiter=",")